In [1]:
import os

In [2]:
%pwd

'c:\\Users\\roysu\\OneDrive\\Documents\\Data-Science-Projects\\Chicken-Disease-MLOps-Project\\research'

In [3]:
##Need to change the directory

In [4]:
##Just go back to main folder/directory

os.chdir("../")
%pwd

'c:\\Users\\roysu\\OneDrive\\Documents\\Data-Science-Projects\\Chicken-Disease-MLOps-Project'

In [5]:
##Update Entity --> To update the Return type of a function
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    ##This data is coming from "config.yaml"
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from chicken_disease_classifier.constants import *
from chicken_disease_classifier.utils.common import read_yaml, create_directories



In [7]:
##Update the configuration manager in src config
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [8]:
import os
import urllib.request as request
import zipfile
from chicken_disease_classifier import logger
from chicken_disease_classifier.utils.common import get_size

In [9]:
##Update the components
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [10]:
##Update the pipeline 
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-01-24 12:54:37,177: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-01-24 12:54:37,184: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-24 12:54:37,184: INFO: common: created directory at: artifacts]
[2024-01-24 12:54:37,192: INFO: common: created directory at: artifacts/data_ingestion]
[2024-01-24 12:54:37,196: INFO: 193906067: File already exists of size: ~ 11345 KB]


In [11]:
##After running the above code, you will see Artifacts folder is created

In [12]:
##Now convert this Jupyter notebook into a Modular coding